In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import os

import warnings
warnings.filterwarnings("ignore")


In [2]:
PATH_TO_DATA = './data'
if not os.path.exists(PATH_TO_DATA):
    os.mkdir(PATH_TO_DATA)

if not os.path.exists('./data/cats_and_dogs_filtered'):
    !curl -O https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
    !unzip cats_and_dogs_filtered.zip -d {PATH_TO_DATA}
PATH_TO_DATA = os.path.join(PATH_TO_DATA, 'cats_and_dogs_filtered')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.4M  100 65.4M    0     0  24.8M      0  0:00:02  0:00:02 --:--:-- 24.8M
Archive:  cats_and_dogs_filtered.zip
   creating: ./data/cats_and_dogs_filtered/
  inflating: ./data/cats_and_dogs_filtered/vectorize.py  
   creating: ./data/cats_and_dogs_filtered/validation/
   creating: ./data/cats_and_dogs_filtered/train/
   creating: ./data/cats_and_dogs_filtered/validation/dogs/
  inflating: ./data/cats_and_dogs_filtered/validation/dogs/dog.2127.jpg  
  inflating: ./data/cats_and_dogs_filtered/validation/dogs/dog.2126.jpg  
  inflating: ./data/cats_and_dogs_filtered/validation/dogs/dog.2125.jpg  
  inflating: ./data/cats_and_dogs_filtered/validation/dogs/dog.2124.jpg  
  inflating: ./data/cats_and_dogs_filtered/validation/dogs/dog.2123.jpg  
  inflating: ./data/cats_and_dogs_filtered/validation/dogs/dog.2122.jpg  
  inflating: ./

In [3]:
normalizer = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # default p=0.5
    transforms.ToTensor(),
    normalizer
])


dataset = ImageFolder(PATH_TO_DATA, transform = train_transform)
train_samples, val_samples = int(.90 * len(dataset)), len(dataset)-int(.90 * len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, lengths = [train_samples,val_samples])


print(len(train_dataset))


2700


In [4]:
class AlexNet(nn.Module):
  def __init__(self, classes=2, dropout_p=0.5):
    super(AlexNet, self).__init__()
    self.classes = classes
    self.dropout_p = dropout_p


    self.feature_extractor = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=64),

        nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=192),

        nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1 ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=384),

        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(num_features=256),

        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=256),


    )
    self.avgpool = nn.AdaptiveAvgPool2d((2,2))

    self.head = nn.Sequential(
        nn.Dropout(p=self.dropout_p),
        nn.Linear(in_features=256*2*2, out_features=4096),
        nn.ReLU(),

        nn.Dropout(p=self.dropout_p),
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(),

        nn.Linear(in_features=4096, out_features=self.classes)

    )

  def forward(self, x):
      batch_size = x.shape[0]
      x = self.feature_extractor(x)
      x = self.avgpool(x)
      x = x.reshape(batch_size, -1)
      x = self.head(x)

      return x

model = AlexNet()
rand = torch.rand(4,3, 500,500)
model(rand)


tensor([[-0.0696,  0.0206],
        [-0.0645,  0.1141],
        [-0.0283,  0.0299],
        [-0.0016, -0.0566]], grad_fn=<AddmmBackward0>)

In [12]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)
model = AlexNet()
model = model.to(DEVICE)

EPOCHS = 30
optimizer = optim.Adam(params=model.parameters(), lr= 0.0001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)


def train(model, DEVICE, epochs, loss_fn, batch_size, trainloader, valloader):

    log_training = {"epochs": [],
                    "training_loss": [],
                    "training_acc": [],
                    "validation_loss":[],
                    "validation_acc":[]}

    for epoch in range(1, EPOCHS+1):
           print(f"Starting Epoch {epoch}")

           training_losses, training_accuracies= [], []
           validation_losses, validation_accuracies = [], []

           model.train()
           for images,labels in tqdm(trainloader):
              images = images.to(DEVICE)
              labels = labels.to(DEVICE)

              out = model(images)
              loss = loss_fn(out, labels)
              training_losses.append(loss.item())

              predictions = torch.argmax(out, dim=1)
              accuracy = torch.mean((predictions == labels).float())
              training_accuracies.append(accuracy.item())

              optimizer.zero_grad()
              loss.backward()
              optimizer.step()


           model.eval()
           for images,labels in tqdm(valloader):
              images = images.to(DEVICE)
              labels = labels.to(DEVICE)
              with torch.no_grad():
                out = model(images)

              loss = loss_fn(out, labels)
              validation_losses.append(loss.item())

              predictions = torch.argmax(out, dim=1)
              accuracy = torch.mean((predictions == labels).float())
              validation_accuracies.append(accuracy.item())


           training_loss_mean, training_acc_mean = np.mean(training_losses), np.mean(training_accuracies)
           validation_loss_mean, validation_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

           log_training['epochs'].append(epoch)
           log_training['training_loss'].append(training_loss_mean)
           log_training['training_acc'].append(training_acc_mean)
           log_training['validation_loss'].append(validation_loss_mean)
           log_training['validation_acc'].append(validation_acc_mean)


           print("Training Loss", training_loss_mean)
           print("Training Acc", training_acc_mean)
           print("Validation Loss", validation_loss_mean)
           print("Validation Acc", validation_acc_mean)

    return log_training, model

logs, model = train(model, DEVICE, EPOCHS, loss_fn, batch_size, trainloader, valloader)



cuda
Starting Epoch 1


  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Training Loss 0.6815326518790666
Training Acc 0.627906976744186
Validation Loss 0.6611953854560852
Validation Acc 0.6696022748947144
Starting Epoch 2


  0%|          | 0/43 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Training Loss 0.23501615676768992
Training Acc 0.8995881787566251
Validation Loss 1.3690006256103515
Validation Acc 0.5485795497894287
Starting Epoch 30


  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Training Loss 0.2212075620543125
Training Acc 0.9083091089891833
Validation Loss 1.3719053983688354
Validation Acc 0.531534093618393
